In [1]:
import os

import dill

from collab.utils import find_repo_root

In [2]:
class ProgressSaverContext:
    def __init__(self, file_path):
        self.file_path = file_path

    def save_progress(self, name):
        os.makedirs(os.path.dirname(self.file_path), exist_ok=True)
        with open(self.file_path, "wb") as f:
            dill.dump(name, f)

    def load_progress(self):
        with open(self.file_path, "rb") as f:
            return dill.load(f)


def progress_saver(name, subfolder, filename=None, properties=None, code_f=None):
    property_names = [name for name, value in globals().items() if value in properties]

    properties_string = "_".join(
        [f"{var_name}_{var}" for var_name, var in zip(property_names, properties)]
    )
    if filename is None:
        filename = f"{name}_{properties_string}.pkl"

    file_path = os.path.join(find_repo_root(), "data", subfolder, filename)

    print(file_path)

    progress_context = ProgressSaverContext(file_path)

    print("path missing, executing code", not os.path.exists(file_path))
    if not os.path.exists(file_path):
        try:
            result = code_f()
            progress_context.save_progress(result)
            return result
        except Exception as e:
            print(f"Error executing code: {e}")

    else:
        loaded_data = progress_context.load_progress()
        return loaded_data

In [6]:
import shutil

num = 0
blam = 2


def test_var1():
    return 1


def test_var2():
    return 2


file_path = os.path.join(
    find_repo_root(), "data", "test", "test_variable_num_0_blam_2.pkl"
)
folder_path = os.path.dirname(file_path)
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
assert not os.path.exists(folder_path)


test_variable = progress_saver(
    name="test_variable", subfolder="test", properties=[num, blam], code_f=test_var1
)


assert os.path.exists(folder_path)

assert test_variable == 1

with open(file_path, "rb") as f:
    loaded_variable = dill.load(f)

assert loaded_variable == 1


test_variable = progress_saver(
    name="test_variable", subfolder="test", properties=[num, blam], code_f=test_var2
)

print(test_variable)

with open(file_path, "rb") as f:
    loaded_variable = dill.load(f)

assert loaded_variable == 1

# # # print(test_variable)

/home/rafal/s78projects/collaborative-intelligence/data/test/test_variable_num_0_blam_2.pkl
path missing, executing code True
/home/rafal/s78projects/collaborative-intelligence/data/test/test_variable_num_0_blam_2.pkl
path missing, executing code False
1
